## Example 1 - Simple

In [5]:
import time
import numpy
import pyaudio
import fluidsynth
from scipy import stats

pa = pyaudio.PyAudio()
strm = pa.open(
    format = pyaudio.paInt16,
    channels = 2, 
    rate = 44100, 
    output = True)

s = []

fl = fluidsynth.Synth()

# Initial silence is 1 second
s = numpy.append(s, fl.get_samples(44100 * 1))

sfid = fl.sfload("../data/sf2/example.sf2")
fl.program_select(0, sfid, 0, 0)

fl.noteon(0, 60, 30)
fl.noteon(0, 67, 30)
fl.noteon(0, 76, 30)

fl.noteon(0, 50, 30)
fl.noteon(0, 57, 30)
fl.noteon(0, 66, 30)

# Chord is held for 2 seconds
s = numpy.append(s, fl.get_samples(44100 * 2))

print(stats.describe(s))

fl.noteoff(0, 60)
fl.noteoff(0, 67)
fl.noteoff(0, 76)

# Decay of chord is held for 1 second
s = numpy.append(s, fl.get_samples(44100 * 0))

print(stats.describe(s))

fl.delete()

samps = fluidsynth.raw_audio_string(s)

print (len(samps))
print ('Starting playback')
strm.write(samps)


DescribeResult(nobs=264600, minmax=(-3408.0, 3597.0), mean=0.2535449735449735, variance=386313.05661085196, skewness=0.001675100241302679, kurtosis=1.6251365859280398)
DescribeResult(nobs=264600, minmax=(-3408.0, 3597.0), mean=0.2535449735449735, variance=386313.05661085196, skewness=0.001675100241302679, kurtosis=1.6251365859280398)
529200
Starting playback


## Example 2 - Sequencer

In [6]:
#!/usr/bin/env python3

import time
import fluidsynth
from ctypes import *

seqduration = 1000

def schedule_next_callback():
    # I want to be called back before the end of the next sequence
    
    callbackdate = int(now + seqduration/2)
    print("callback called: " + str(callbackdate))
    sequencer.timer(callbackdate, dest=mySeqID)

def schedule_next_sequence():
    global now
    # the sequence to play
    # the beat : 2 beats per sequence
    sequencer.note(int(now + seqduration * 1/4), 0, 60, duration=300, velocity=80, dest=synthSeqID)
    sequencer.note(int(now + seqduration * 2/4), 0, 80, duration=300, velocity=80, dest=synthSeqID)
    sequencer.note(int(now + seqduration * 3/4), 0, 85, duration=500, velocity=80, dest=synthSeqID)
    # melody
    sequencer.note(int(now + seqduration*0/4), 1, 45, duration=int(1000*3/10), velocity=int(127*2/3), dest=synthSeqID)
    sequencer.note(int(now + seqduration*1/4), 1, 50, duration=int(1000*3/10), velocity=int(127*2/3), dest=synthSeqID)
    sequencer.note(int(now + seqduration*2/4), 1, 55, duration=int(1000*3/10), velocity=int(127*3/3), dest=synthSeqID)
    sequencer.note(int(now + seqduration*3/4), 1, 50, duration=int(1000*3/10), velocity=int(127*3/3), dest=synthSeqID)

    # so that we are called back early enough to schedule the next sequence
    # schedule_next_callback()
    # print(sequencer)
    
    now = now + seqduration

def seq_callback(time, event, seq, data):
    schedule_next_sequence()

if __name__=="__main__":
    global sequencer, fs, mySeqID, synthSeqID, now
    fs = fluidsynth.Synth()
    fs.start(driver="coreaudio", midi_driver="coremidi")

    sfid = fs.sfload("../data/sf2/generaluser.v.1.471.sf2")
    fs.program_select(0, sfid, 0, 40) # Guitar 
    fs.program_select(1, sfid, 1, 56) # Trumpet
    # Volume changes
    fs.cc(chan=0, ctrl=7, val=100)
    fs.cc(chan=1, ctrl=7, val=80)

    sequencer = fluidsynth.Sequencer()
    synthSeqID = sequencer.register_fluidsynth(fs)
    mySeqID = sequencer.register_client("mycallback", seq_callback)
    now = sequencer.get_tick()
    schedule_next_sequence()

    time.sleep(10)

    sequencer.delete()
    fs.delete()